# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from contextlib import redirect_stdout
import numpy as np
import os
import pandas as pd
import plotly.express as px

from cost import *
from plotting import *
from prices import *
from inflation import *
from regression import *
from utils import *

In [3]:
estimation_method = 'hardware-capex-energy'  # hardware-capex-energy, hardware-acquisition, cloud
compute_threshold_method = 'top_n'  # top_n, window_percentile
compute_threshold = 10  # e.g. 10 to select top 10; 75 to select top 25%
variant = 'original'  # whatever else distinguishes this run, e.g. 'excluding-AlphaGo'
exclude_models_containing = []  # ['GNMT', 'AlphaZero', 'AlphaGo Master', 'AlphaGo Zero']

estimation_method_lookup = {
    'hardware-capex-energy': estimate_hardware_capex_energy,
    'hardware-acquisition': estimate_hardware_acquisition_cost,
    'cloud': estimate_cloud_costs,
}
cost_estimation_function = estimation_method_lookup[estimation_method]

results_dir = f'results/{estimation_method}-{compute_threshold_method}={compute_threshold}-{variant}/'
os.makedirs(results_dir, exist_ok=True)

# Load data

In [4]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation(
    compute_threshold_method=compute_threshold_method, compute_threshold=compute_threshold,
)

In [5]:
len(frontier_pcd_df), len(hardware_df), len(price_df)

(64, 5510, 145)

# Cost estimation

In [6]:
with open(f'{results_dir}/cost_estimation.out', 'w') as f:
    with redirect_stdout(f):
        cost_df = cost_estimation_function(frontier_pcd_df, hardware_df, price_df)

In [7]:
if estimation_method == 'hardware-capex-energy':
    frontier_pcd_df_copy = frontier_pcd_df.copy()
    with open(f'{results_dir}/component_cost_estimation.out', 'w') as f:
        with redirect_stdout(f):
            component_cost_df = cost_estimation_function(frontier_pcd_df_copy, hardware_df, price_df, separate_components=True)

In [8]:
cost_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Training compute upper bound,Training chip-hours,Code accessibility,Dataset accessibility,Accessibility notes,Organization categorization (from Organization),Possibly over 1e23 FLOP,Training cost trends 2,Training cost trends 3,Cost
88,Gemini 1.0 Ultra,"Multimodal,Language,Vision","Language modelling,Visual question answering,C...",Gemini Team,SOTA improvement,""" Evaluation on a broad range of benchmarks sh...",Hosted access (no API),https://storage.googleapis.com/deepmind-media/...,633.0,Gemini: A Family of Highly Capable Multimodal ...,...,NaN,132000000.0,NaN,NaN,NaN,Industry,NaN,NaN,NaN,2.982734e+07
104,Inflection-2,Language,Language modelling,NaN,Significant use,Inflection-2 either already powers Pi or soon ...,Hosted access (no API),https://inflection.ai/inflection-2,NaN,Inflection-2: The Next Step Up,...,NaN,NaN,NaN,NaN,"via Pi, no API",Industry,NaN,NaN,NaN,1.293179e+07
130,Grok-1,Language,"Language modelling,Chat",NaN,SOTA improvement,"""On these benchmarks, Grok-1 displayed strong ...",Open source,"https://x.ai/model-card/, https://x.ai/blog/gr...",NaN,Announcing Grok,...,7.0,NaN,Unreleased,Unreleased,apache 2.0,Industry,checked,NaN,NaN,NaN
193,Falcon-180B,Language,Language modelling,"Ebtesam Almazrouei, Hamza Alobeidli, Abdulaziz...",SOTA improvement,"""It's currently at the top of the Hugging Face...",Open access (restricted use),https://falconllm.tii.ae/falcon-180b.html; htt...,86.0,The Falcon Series of Open Language Models,...,NaN,17694720.0,NaN,NaN,"""Falcon 180b can be commercially used but unde...",Government,checked,NaN,NaN,1.027732e+07
243,Claude 2,Language,"Language modelling,Chat",NaN,Historical significance,NaN,API access,"https://www.anthropic.com/index/claude-2, http...",0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179,Xception,Vision,Image classification,François Chollet,Highly cited,NaN,NaN,https://arxiv.org/abs/1610.02357,11578.0,Xception: Deep Learning with Depthwise Separab...,...,NaN,43200.0,NaN,NaN,NaN,Industry,NaN,NaN,NaN,1.155451e+04
1180,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,Hosted access (no API),https://arxiv.org/abs/1609.08144,6196.0,Google's Neural Machine Translation System: Br...,...,NaN,655730.0,NaN,NaN,presumably deployed via Google translate,Industry,NaN,NaN,NaN,1.774592e+05
1222,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14887.0,Mastering the game of Go with deep neural netw...,...,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN,NaN,NaN
1226,ResNet-152 (ImageNet),Vision,Image classification,"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.03385,156882.0,Deep Residual Learning for Image Recognition,...,NaN,NaN,NaN,NaN,NaN,Industry,NaN,NaN,NaN,NaN


In [9]:
cost_df['Cost'].notna().sum()

41

In [10]:
cost_df.dropna(subset=['Cost'])['Training time (hours)'].notna().sum()

27

In [11]:
cost_df.dropna(subset=['Cost'])['Hardware utilization'].notna().sum()

17

Exclusion

In [12]:
cost_df[['System', 'Publication date']].tail(15)

,System,Publication date
1045,ResNeXt-101 32x48d,2018-05-02
1076,AlphaZero,2017-12-05
1092,AlphaGo Zero,2017-10-18
1114,OpenAI TI7 DOTA 1v1,2017-08-11
1126,JFT,2017-07-10
1148,MoE,2017-01-23
1151,Libratus,2017-01-01
1152,AlphaGo Master,2017-01-01
1168,PolyNet,2016-11-17
1171,NASv3 (CIFAR-10),2016-11-05


In [13]:
for kw in exclude_models_containing:
    cost_df = cost_df[cost_df['System'].str.contains(kw) == False]
cost_df[['System', 'Publication date']].tail(15)

,System,Publication date
1045,ResNeXt-101 32x48d,2018-05-02
1076,AlphaZero,2017-12-05
1092,AlphaGo Zero,2017-10-18
1114,OpenAI TI7 DOTA 1v1,2017-08-11
1126,JFT,2017-07-10
1148,MoE,2017-01-23
1151,Libratus,2017-01-01
1152,AlphaGo Master,2017-01-01
1168,PolyNet,2016-11-17
1171,NASv3 (CIFAR-10),2016-11-05


Use the below to check data availability for specific systems

In [14]:
# system = 'WizardLM-7B'
# row = cost_df.loc[cost_df['System'] == system]
# print('Cost:', row['Cost'].values[0])
# print('Training hardware:', row['Training hardware'].values[0])
# print('Training time (hours):', row['Training time (hours)'].values[0])
# print('Hardware quantity:', row['Hardware quantity'].values[0])
# print('Hardware utilization:', row['Hardware utilization'].values[0])

# Apply inflation adjustment

In [15]:
cost_df['Cost'].dropna()

88      2.982734e+07
104     1.293179e+07
193     1.027732e+07
294     4.816068e+06
336     4.015593e+07
396     4.534422e+06
408     8.833166e+05
449     8.042599e+05
477     3.384675e+05
514     7.202773e+05
524     2.897985e+06
546     2.224292e+05
576     5.246842e+05
580     5.975319e+05
622     3.590762e+06
629     5.674315e+05
650     8.219443e+04
672     8.940160e+04
680     8.287237e+04
718     5.211218e+04
734     1.345841e+05
737     1.224800e+05
799     1.010278e+05
806     2.107048e+06
841     5.298256e+04
846     1.968088e+05
873     1.180148e+05
877     7.087411e+04
891     6.216056e+05
892     1.070241e+05
917     8.276500e+04
1003    4.799969e+03
1076    2.126712e+05
1092    5.674602e+05
1126    1.596084e+04
1148    3.538189e+03
1152    4.317229e+05
1168    5.636840e+02
1179    1.155451e+04
1180    1.774592e+05
1228    1.854566e+02
Name: Cost, dtype: float64

In [16]:
cost_df = adjust_column_for_inflation(cost_df, 'Cost', 'data/PCU518210518210.csv', '2023-12-01')

In [17]:
cost_df['Cost (inflation-adjusted)'].dropna()

88      2.982734e+07
104     1.296196e+07
193     1.034091e+07
294     4.865570e+06
336     4.058659e+07
396     4.625551e+06
408     9.010687e+05
449     8.202966e+05
477     3.448529e+05
514     7.316676e+05
524     2.945950e+06
546     2.299500e+05
576     5.414374e+05
580     6.166111e+05
622     3.704291e+06
629     5.864088e+05
650     8.479979e+04
672     9.245338e+04
680     8.570126e+04
718     5.384428e+04
734     1.396636e+05
737     1.271026e+05
799     1.053921e+05
806     2.211639e+06
841     5.556354e+04
846     2.067604e+05
873     1.257581e+05
877     7.552439e+04
891     6.629848e+05
892     1.141485e+05
917     8.907265e+04
1003    5.170457e+03
1076    2.299186e+05
1092    6.134806e+05
1126    1.723959e+04
1148    3.863735e+03
1152    4.714453e+05
1168    6.155480e+02
1179    1.261763e+04
1180    1.937871e+05
1228    2.063058e+02
Name: Cost (inflation-adjusted), dtype: float64

In [18]:
assert cost_df['Cost (inflation-adjusted)'].notna().sum() == cost_df['Cost'].notna().sum()

# Regression

In [19]:
cost_df['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(cost_df['Publication date']))

In [20]:
reg_results = fit_ols_regression(cost_df, ['Publication date (float)'], 'Cost (inflation-adjusted)', logy=True)
reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.582
Model:                            OLS   Adj. R-squared:                  0.571
Method:                 Least Squares   F-statistic:                     54.24
Date:                Mon, 09 Dec 2024   Prob (F-statistic):           6.79e-09
Time:                        14:37:02   Log-Likelihood:                -44.783
No. Observations:                  41   AIC:                             93.57
Df Residuals:                      39   BIC:                             96.99
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -765.4884    104.668     -7.313      0.000    -977.200    -553.776
x1             0.3815      0.052      7.365      0.000       0.277       0.486
==============================================================================
Omnibus:                        1.604   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.449   Jarque-Bera (JB):                1.552
Skew:                           0.415   Prob(JB):                        0.460
Kurtosis:                       2.531   Cond. No.                     1.83e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.83e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [21]:
with open(f'{results_dir}/regression_results.out', 'w') as f:
    with redirect_stdout(f):
        print_growth_rates(reg_results, round_digits=None)
print_growth_rates(reg_results, ci=90, round_digits=5)

N=41.0
R^2=0.58
0.38152 OOMs/year (90% CI: 0.29424, 0.4688)
2.40727x/year (90% CI: 1.969x, 2.9431x)
doubling time of 9.46822 months (90% CI: 7.70547, 12.27672)


In [22]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [23]:

predicted_cost_df = get_predictions(reg_results, pred_years, ['Publication date (float)'])
predicted_cost_df['Publication date'] = predicted_cost_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_cost_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,3.283870,0.309723,2.762024,3.805715,1.932911,4.634828,2015.00000,2015-01-01
1,3.322407,0.304875,2.808732,3.836083,1.974584,4.670231,2015.10101,2015-02-06
2,3.360945,0.300039,2.855417,3.866473,2.016206,4.705684,2015.20202,2015-03-15
3,3.399483,0.295216,2.902080,3.896886,2.057777,4.741189,2015.30303,2015-04-21
4,3.438021,0.290408,2.948719,3.927322,2.099297,4.776745,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,6.944965,0.239413,6.541583,7.348347,5.635201,8.254729,2024.59596,2024-08-06
96,6.983503,0.244010,6.572377,7.394629,5.671333,8.295673,2024.69697,2024-09-12
97,7.022041,0.248631,6.603128,7.440954,5.707410,8.336671,2024.79798,2024-10-19
98,7.060579,0.253277,6.633839,7.487318,5.743433,8.377724,2024.89899,2024-11-25


In [24]:
predicted_cost_df.to_csv(results_dir + 'predicted_cost_dataset.csv', index=False)

# Export data

In [25]:
keep_cols = [
    'System',
    'Domain',
    'Task',
    'Model accessibility',
    'Reference',
    'Publication date',
    'Organization',
    'Parameters',
    'Training compute (FLOP)',
    'Training dataset size (datapoints)',
    'Epochs',
    'Training time (hours)',
    'Training hardware',
    'Country (from Organization)',
    'Base model',
    'Finetune compute (FLOP)',
    'Hardware quantity',
    'Hardware utilization',
    'Training cloud compute vendor',
    'Training data center',
    # 'Training time (chip hours)',
    'Cost',
    'Cost (inflation-adjusted)',
]
cost_df[keep_cols]

,System,Domain,Task,Model accessibility,Reference,Publication date,Organization,Parameters,Training compute (FLOP),Training dataset size (datapoints),...,Training hardware,Country (from Organization),Base model,Finetune compute (FLOP),Hardware quantity,Hardware utilization,Training cloud compute vendor,Training data center,Cost,Cost (inflation-adjusted)
88,Gemini 1.0 Ultra,"Multimodal,Language,Vision","Language modelling,Visual question answering,C...",Hosted access (no API),Gemini: A Family of Highly Capable Multimodal ...,2023-12-06,Google DeepMind,NaN,5.000000e+25,NaN,...,Google TPU v4,Multinational,NaN,NaN,55000.0,NaN,NaN,NaN,2.982734e+07,2.982734e+07
104,Inflection-2,Language,Language modelling,Hosted access (no API),Inflection-2: The Next Step Up,2023-11-22,Inflection AI,NaN,1.001000e+25,NaN,...,NVIDIA H100 SXM5,United States of America,NaN,NaN,5000.0,NaN,NaN,NaN,1.293179e+07,1.296196e+07
130,Grok-1,Language,"Language modelling,Chat",Open source,Announcing Grok,2023-11-04,xAI,3.140000e+11,2.900000e+24,NaN,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,Falcon-180B,Language,Language modelling,Open access (restricted use),The Falcon Series of Open Language Models,2023-09-06,Technology Innovation Institute,1.800000e+11,3.760000e+24,3.500000e+12,...,NVIDIA A100 SXM4 40 GB,United Arab Emirates,NaN,NaN,4096.0,0.1876,Amazon Web Services,NaN,1.027732e+07,1.034091e+07
243,Claude 2,Language,"Language modelling,Chat",API access,NaN,2023-07-11,Anthropic,NaN,3.866000e+24,NaN,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179,Xception,Vision,Image classification,NaN,Xception: Deep Learning with Depthwise Separab...,2016-10-07,Google,2.285595e+07,4.360000e+20,3.500000e+08,...,NVIDIA Tesla K80,United States of America,NaN,NaN,60.0,NaN,NaN,NaN,1.155451e+04,1.261763e+04
1180,GNMT,Language,Translation,Hosted access (no API),Google's Neural Machine Translation System: Br...,2016-09-26,Google,2.780000e+08,6.620000e+21,1.230000e+09,...,NVIDIA Tesla K80,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,1.774592e+05,1.937871e+05
1222,AlphaGo Lee,Games,Go,NaN,Mastering the game of Go with deep neural netw...,2016-01-27,DeepMind,NaN,1.900000e+21,2.940000e+07,...,NaN,United Kingdom of Great Britain and Northern I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1226,ResNet-152 (ImageNet),Vision,Image classification,NaN,Deep Residual Learning for Image Recognition,2015-12-10,Microsoft,6.000000e+07,1.210000e+19,1.280000e+06,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
cost_df[keep_cols].to_csv(results_dir + 'cost_dataset.csv', index=False)

# Plots

In [27]:
fig = px.scatter(
    cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)
fig.update_traces(textposition='top center')

# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD, log scale)')

# title
fig.update_layout(title_text=get_cost_plot_title(estimation_method, compute_threshold_method, compute_threshold))

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    )
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

In [28]:
label_systems = ['GNMT', 'AlphaGo Master', 'AlphaGo Zero', 'AlphaZero', 'DALL-E', 'GPT-3 175B (davinci)', 'PaLM (540B)', 'Llama 2-70B', 'Falcon 180B', 'GPT-4', 'Gemini 1.0 Ultra', 'Inflection-2']

tpu_mask = cost_df['Training hardware'].str.contains('TPU', na=False)
tpu_cost_df = cost_df.loc[tpu_mask]
gpu_cost_df = cost_df.loc[~tpu_mask]

fig = px.scatter(
    gpu_cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    log_y=True,
)
fig.add_scatter(
    x=tpu_cost_df['Publication date'],
    y=tpu_cost_df['Cost (inflation-adjusted)'],
    mode='markers',
    marker_symbol='circle-open' if estimation_method != 'cloud' else 'circle',
    name='Using estimated cost of TPU' if estimation_method != 'cloud' else '',
    showlegend=estimation_method != 'cloud',
)
fig.add_scatter(
    x=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['Publication date'],
    y=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['Cost (inflation-adjusted)'],
    text=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['System'],
    mode='text',
    showlegend=False,
)
fig.add_scatter(
    x=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['Publication date'],
    y=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['Cost (inflation-adjusted)'],
    text=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['System'],
    mode='text',
    showlegend=False,
)

# Marker color
fig.update_traces(
    marker=dict(
        color='rgb(0,100,200)',
    ),
    selector=dict(mode='markers'),
)

# Shade in CI
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='95% CI of mean',
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# axis limits
fig.update_xaxes(range=[pred_start_date, pred_end_date])
# fig.update_xaxes(range=['2015-01-01', '2025-01-01'])  # manual
if estimation_method == 'hardware-acquisition':
    fig.update_yaxes(range=[4, 10])
else:
    fig.update_yaxes(range=[1, 9])

# legend on bottom-right of the axes
fig.update_layout(
    legend=dict(
        x=0.45,
        y=0.05,
    )
)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD, log scale)')

# title
fig.update_layout(title_text=get_cost_plot_title(estimation_method, compute_threshold_method, compute_threshold))

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    ),
    title_x=0.5,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=60, b=10))

save_plot(fig, results_dir, 'cost_regression')

fig.show()

# Cost components

In [29]:
cost_component_names = [
    'AI accelerator chip cost',
    'Other server components cost',
    'Cluster-level interconnect cost',
    'Energy cost',
]

In [30]:
for key in cost_component_names:
    component_cost_df[f"{key} (%)"] = component_cost_df[key] / component_cost_df['Cost'] * 100
component_cost_df['AI accelerator chip cost (%)']

88      44.541490
104     46.631554
130           NaN
193     43.637309
243           NaN
          ...    
1179    43.829051
1180    43.894050
1222          NaN
1226          NaN
1228    34.484219
Name: AI accelerator chip cost (%), Length: 64, dtype: float64

In [31]:
cost_component_pc_names = [name + ' (%)' for name in cost_component_names]
filtered_component_cost_df = component_cost_df.dropna(subset=cost_component_pc_names).sort_values(by='Publication date')

In [32]:
# Stacked bar chart of cost components, using component_cost_df
fig = px.bar(
    filtered_component_cost_df,
    x='System',
    y=cost_component_pc_names,
    barmode='stack',
)

# axis labels
fig.update_xaxes(title_text='ML model')
fig.update_yaxes(title_text='% of amortized hardware CapEx + energy')
fig.update_layout(
    legend=dict(
        title_text='Cost component',
        x=0.60,
        y=0.05,
    )
)
# limits 0 to 100
fig.update_yaxes(range=[0, 100])

fig.update_yaxes(tickvals=list(range(0, 101, 10)))

# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_component_percentage')

fig.show()

In [33]:
# Average percentage for each component
filtered_component_cost_df[cost_component_pc_names].mean()

AI accelerator chip cost (%)           44.401345
Other server components cost (%)       28.980725
Cluster-level interconnect cost (%)    17.213078
Energy cost (%)                         9.404852
dtype: float64

In [34]:
fig = px.bar(
    filtered_component_cost_df,
    x='System',
    y='Energy cost (%)',
    barmode='stack',
    # labels='Cost %',
    # text='Energy cost %',
)
# axis labels
fig.update_xaxes(title_text='System')
fig.update_yaxes(title_text='Energy cost (% of amortized hardware CapEx + energy)')
# fig.update_layout(
#     legend=dict(
#         title_text='Cost component',
#         x=0.75,
#         y=0.05,
#     )
# )
# limits 0 to 100
fig.update_yaxes(range=[0, 30])
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_percentage')

fig.show()

In [35]:
fig = px.scatter(
    filtered_component_cost_df,
    x='Publication date',
    y='Energy cost',
    text='System',
)
# axis labels
fig.update_xaxes(title_text='System')
fig.update_yaxes(title_text='Energy cost')
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_cost')

fig.show()

In [36]:
from energy import energy_price

# Stacked bar chart of cost components, using component_cost_df
filtered_component_cost_df.loc[:, 'Energy (kWh)'] = [
    row['Energy cost'] / energy_price(row['Publication date'].year) 
    for _, row in filtered_component_cost_df.iterrows()
]
fig = px.scatter(
    filtered_component_cost_df,
    x='Publication date',
    y='Energy (kWh)',
    text='System',
)
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_kwh')

fig.show()

In [37]:
filtered_component_cost_df.columns

Index(['System', 'Domain', 'Task', 'Authors', 'Notability criteria',
       'Notability criteria notes', 'Model accessibility', 'Link', 'Citations',
       'Reference', 'Publication date', 'Organization', 'Parameters',
       'Parameters notes', 'Training compute (FLOP)', 'Training compute notes',
       'Training dataset', 'Training dataset notes',
       'Training dataset size (datapoints)', 'Dataset size notes', 'Epochs',
       'Inference compute (FLOP)', 'Inference compute notes',
       'Training time (hours)', 'Training time notes', 'Training hardware',
       'Approach', 'Training compute cost (2020 USD)', 'Compute cost notes',
       'Compute sponsor categorization', 'Confidence', 'Abstract',
       'Last modified', 'Created By', 'Benchmark data', 'Exclude',
       'Country (from Organization)', 'Base model', 'Finetune compute (FLOP)',
       'Finetune compute notes', 'Hardware quantity', 'Hardware utilization',
       'Training cost trends', 'Training cloud compute vendor',
 

In [38]:
filtered_component_cost_df = filtered_component_cost_df.dropna(subset=['Training hardware'])
power_col = 'Power capacity for final training run (kW)'
filtered_component_cost_df.loc[:, power_col] = [
    cluster_power_capacity(row['Training hardware'], row['Hardware quantity'], hardware_df, row['Organization'])
    for _, row in filtered_component_cost_df.iterrows()
]

fig = px.scatter(
    filtered_component_cost_df,
    x='Publication date',
    y=power_col,
    text='System',
)
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'power_capacity_kw')

fig.show()

In [39]:
filtered_component_cost_df['Publication date (float)'] = datetime_to_float_year(
    pd.to_datetime(filtered_component_cost_df['Publication date'])
)

In [40]:
power_reg_results = fit_ols_regression(
    filtered_component_cost_df,
    ['Publication date (float)'],
    power_col,
    logy=True
)
power_reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.723
Model:                            OLS   Adj. R-squared:                  0.714
Method:                 Least Squares   F-statistic:                     80.75
Date:                Mon, 09 Dec 2024   Prob (F-statistic):           3.86e-10
Time:                        14:37:07   Log-Likelihood:                -21.075
No. Observations:                  33   AIC:                             46.15
Df Residuals:                      31   BIC:                             49.14
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -696.4603     77.804     -8.951      0.000    -855.143    -537.778
x1             0.3460      0.039      8.986      0.000       0.267       0.425
==============================================================================
Omnibus:                        3.576   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.167   Jarque-Bera (JB):                2.667
Skew:                          -0.165   Prob(JB):                        0.264
Kurtosis:                       4.353   Cond. No.                     1.91e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.91e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [41]:
with open(f'{results_dir}/power_regression_results.out', 'w') as f:
    with redirect_stdout(f):
        print_growth_rates(power_reg_results)
print_growth_rates(power_reg_results)

N=33.0
R^2=0.72
0.34601445632359445 OOMs/year (90% CI: 0.28072805333693446, 0.41130085931025445)
2.218270257945275x/year (90% CI: 1.9086577203681487x, 2.578106532550776x)
doubling time of 10.439910477582696 months (90% CI: 8.782767811440145, 12.86782672778399)


In [42]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [43]:
predicted_power_df = get_predictions(power_reg_results, pred_years, ['Publication date (float)'])
predicted_power_df['Publication date'] = predicted_power_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_power_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,0.758796,0.231188,0.366813,1.150779,-0.133576,1.651168,2015.00000,2015-01-01
1,0.793747,0.227557,0.407919,1.179574,-0.095939,1.683432,2015.10101,2015-02-06
2,0.828698,0.223936,0.449011,1.208385,-0.058342,1.715737,2015.20202,2015-03-15
3,0.863649,0.220323,0.490087,1.237210,-0.020786,1.748084,2015.30303,2015-04-21
4,0.898600,0.216720,0.531147,1.266053,0.016727,1.780472,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,4.079136,0.174135,3.783888,4.374385,3.224825,4.933448,2024.59596,2024-08-06
96,4.114087,0.177572,3.813011,4.415164,3.257744,4.970431,2024.69697,2024-09-12
97,4.149038,0.181027,3.842104,4.455973,3.290618,5.007459,2024.79798,2024-10-19
98,4.183989,0.184500,3.871167,4.496812,3.323446,5.044533,2024.89899,2024-11-25


## Power plot

In [44]:
label_systems = ['GNMT', 'AlphaGo Master', 'AlphaGo Zero', 'AlphaZero', 'DALL-E', 'GPT-3 175B (davinci)', 'PaLM (540B)', 'Llama 2-70B', 'Falcon 180B', 'GPT-4', 'Gemini 1.0 Ultra', 'Inflection-2']

fig = px.scatter(
    filtered_component_cost_df,
    x='Publication date',
    y=power_col,
    log_y=True,
)

# Marker color
fig.update_traces(
    marker=dict(
        color='rgb(0,100,200)',
    ),
    selector=dict(mode='markers'),
)

fig.add_scatter(
    x=filtered_component_cost_df.loc[filtered_component_cost_df['System'].isin(label_systems)]['Publication date'],
    y=filtered_component_cost_df.loc[filtered_component_cost_df['System'].isin(label_systems)][power_col],
    text=filtered_component_cost_df.loc[filtered_component_cost_df['System'].isin(label_systems)]['System'],
    mode='text',
    showlegend=False,
)

# Shade in CI
fig.add_scatter(
    x=predicted_power_df['Publication date'],
    y=10**predicted_power_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_power_df['Publication date'],
    y=10**predicted_power_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='95% CI of mean',
)
fig.add_scatter(
    x=predicted_power_df['Publication date'],
    y=10**predicted_power_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**power_reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# axis limits
fig.update_xaxes(range=[pred_start_date, pred_end_date])
# fig.update_xaxes(range=['2015-01-01', '2025-01-01'])  # manual
# fig.update_yaxes(range=[1, 6])

# legend on bottom-right of the axes
fig.update_layout(
    legend=dict(
        x=0.45,
        y=0.05,
    )
)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Power (kW, log scale)')

# title
fig.update_layout(title_text='Cluster power required for final training run')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    ),
    title_x=0.5,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=60, b=10))

save_plot(fig, results_dir, 'power_regression')

fig.show()